<a href="https://colab.research.google.com/github/mayonaise01/DADS5001_Data_Product/blob/main/Data_Product_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>